In [8]:
import win32gui

# Definir la función de callback para EnumWindows
def callback(hwnd, hwnds):
    if 'SAP Logon' in win32gui.GetWindowText(hwnd):
        hwnds.append(hwnd)

# Crear una lista para almacenar los identificadores de ventana
hwnds = []

# Llamar a EnumWindows para enumerar todas las ventanas y agregar los identificadores de ventana a la lista
win32gui.EnumWindows(callback, hwnds)

# Obtener la cantidad de ventanas hijas
n_windows = len(hwnds) # Restar 1 para excluir la ventana principal

# Imprimir la cantidad de ventanas hijas
print(n_windows)


-1


In [19]:
import tkinter as tk
import pickle
import os
import sys
import subprocess
import time
from datetime import datetime
from tkinter import *
import win32gui
import win32com.client as win32
import requests
import shutil
import pandas as pd
import win32con
import pywintypes
import win32com.client
import pyperclip



# ======================================== Parte 1 _ Inicializacion ========================================
class InputForm:

    def on_entry_changed(self, input_str):
        
        # ----Esta def permite que solo lleguen a un maximo de 8 caracteres

        if len(input_str) > 10:
            self.radate_input.delete(10, 'end')
            self.createdon_input.delete(10, 'end')
            self.to_input.delete(10, 'end')

    def __init__(self, master):

        print("==============================================================================================================")
        print("====INICIALIZACION DE -REPORTE YRA2-")
        print("==============================================================================================================\n")

        print("==============================================================================================================")
        print("====INICIALIZACION DE LA VENTANA EMERGENTE DE -DESCARGA YRA2-")
        print("==============================================================================================================\n")

        master.attributes('-topmost', True)

        # ----Genera el titulo de la pantalla emergente
        self.master = master
        master.title('INICIO DESCARGA YRA2') #Titulo en el Pop up de ingresar Usuario y contraeña

        # ----El directorio de los datos incriptados del usuario y password
        #directorio_login_bin = (r"C:\Program Files (x86)\Nokia\Reporte YRA2\Reporte_YRA2\login_info.bin")
            # ----Si se quiere ejecutar en el computador  
        directorio_login_bin = os.path.join(os.path.join(os.environ['USERPROFILE']), 'OneDrive - Nokia', 'Archivos personales', 'Automatizacion Python', 'Reporte YRA2 (P20)') + '\\login_info.bin'

        # ----Load saved username and password if they exist
        try:
            with open(directorio_login_bin, 'rb') as f:
                self.login_info = pickle.load(f)
        except:
            self.login_info = {'username': '', 'password': ''}

        # ----Create labels and input fields

        self.username_label = tk.Label(master, text='Fechas = dd.mm.yy', font=('Helvetica', 9, 'bold'))
        self.username_label.grid(row=0, column=1, padx=5, pady=5)
        
        self.username_label = tk.Label(master, text='Username:')
        self.username_label.grid(row=1, column=0, padx=5, pady=5)
        self.username_input = tk.Entry(master, width=16)
        self.username_input.insert(0, self.login_info['username'])
        self.username_input.grid(row=1, column=1, padx=5, pady=5)

        self.password_label = tk.Label(master, text='Password:')
        self.password_label.grid(row=2, column=0, padx=5, pady=5)
        self.password_input = tk.Entry(master, show='*', width=16)
        self.password_input.insert(0, self.login_info['password'])
        self.password_input.grid(row=2, column=1, padx=5, pady=5)

        self.radate_label = tk.Label(master, text='RA Date:')
        self.radate_label.grid(row=3, column=0, padx=5, pady=5)
        self.radate_input = tk.Entry(master, width=16)
        self.radate_input.grid(row=3, column=1, padx=5, pady=5)
        self.radate_input.bind('<KeyRelease>', lambda event: self.on_entry_changed(self.radate_input.get()))

        self.createdon_label = tk.Label(master, text='Created on:')
        self.createdon_label.grid(row=4, column=0, padx=5, pady=5)
        self.createdon_input = tk.Entry(master, width=16)
        self.createdon_input.grid(row=4, column=1, padx=5, pady=5)
        self.createdon_input.bind('<KeyRelease>', lambda event: self.on_entry_changed(self.createdon_input.get()))

        self.to_label = tk.Label(master, text='To:')
        self.to_label.grid(row=4, column=2, padx=5, pady=5)
        self.to_input = tk.Entry(master, width=16)
        self.to_input.grid(row=4, column=3, padx=5, pady=5)
        self.to_input.bind('<KeyRelease>', lambda event: self.on_entry_changed(self.to_input.get()))

        self.wbs_label = tk.Label(master, text='WBS\n(Separar por\n"Saltos de Linea"):')
        self.wbs_label.grid(row=5, column=0, padx=5, pady=5)
        self.wbs_input = tk.Text(master,height=5, width=12)
        self.wbs_input.grid(row=5, column=1, padx=5, pady=5)

        # ----Create submit button
        self.submit_button = tk.Button(master, text='Submit', command=self.submit)
        self.submit_button.grid(row=6, column=1, padx=5, pady=5)


    def submit(self):

        print("========================================================================")
        print("----Se presionó el boton SUBMIT ____ Procede a inicar -open_sap.saplogin(variante,username, password)-")
        print("========================================================================\n")

        # ----Get the values of the input fields and do something with them
        username = self.username_input.get()
        password = self.password_input.get()
        radate = self.radate_input.get()
        createdon = self.createdon_input.get()
        to = self.to_input.get()
        wbs = self.wbs_input.get('1.0', 'end-1c')
        
        # ----Esto es para volverlo lista, pero no se debe activar ya que el portapapleles no lee listas para ejecutar
        wbs_list = wbs.split("\n")or(",")or(" ")or(", ")or("  ")

        # ----Close the window and end the program pero si quieren seguir las varialbles se debe pner return al final del todo
        self.master.destroy()
        
        print("========================================================================================================================")
        print("====FINALIZACION DE LA VENTANA EMERGENTE DE -DESCARGA YRA2-")
        print("========================================================================================================================\n")

        # ----Save the login information
        self.login_info['username'] = username
        self.login_info['password'] = password
       
        # ----El directorio de los datos incriptados del usuario y password
        #directorio_login_bin = (r"C:\Program Files (x86)\Nokia\Reporte YRA2\Reporte_YRA2\login_info.bin")
            # ----Si se quiere ejecutar en el computador        
        directorio_login_bin = os.path.join(os.path.join(os.environ['USERPROFILE']), 'OneDrive - Nokia', 'Archivos personales', 'Automatizacion Python', 'Reporte YRA2 (P20)') + '\\login_info.bin'
        
        with open(directorio_login_bin, 'wb') as f:
            pickle.dump(self.login_info, f)

        #--------------------------------------------------------------------------------------------------------------------
        # <<<<<<<<<SE EJECUTA LA APERTURA DE SAP
        saplogin(username, password, radate, createdon, to, wbs_list) 
        #--------------------------------------------------------------------------------------------------------------------

        # ----Esto permite que se cierre la ventana emergente con self.master.destroy() pero que las variables y la ejecucion de los "def" no se borren
        return
# ==========================================================================================================



# ======================================== Parte 2 _ Ejecucion SAP y el resto de Procesos ========================================
# Definir la función de callback para EnumWindows
def callback(hwnd, hwnds):
    if 'SAP Logon' in win32gui.GetWindowText(hwnd):
        hwnds.append(hwnd)

def saplogin(username, password, radate, createdon, to, wbs_list):

    # ----Crear una lista para almacenar los identificadores de ventana
    hwnds = []
    # ----Llamar a EnumWindows para enumerar todas las ventanas y agregar los identificadores de ventana a la lista
    win32gui.EnumWindows(callback, hwnds)
    # ----Obtener la cantidad de ventanas hijas
    n_windows = len(hwnds) #- 1 # Restar 1 para excluir la ventana principal
    # ----Imprimir la cantidad de ventanas hijas
    print("Cantidad SAP LOGON:", n_windows)

    if n_windows == 0:

        # ----This function will Login to SAP from the SAP Logon window
        print("==============================================================================================================")
        print("====INICIALIZACION DE -SAP LOGIN-")
        print("==============================================================================================================\n")

        # ----Path del ejecutable de SAP
        path = r"C:\Program Files (x86)\SAP\SAPGUI770\SAPgui\saplogon.exe"

        # ----Esperar a que abra la pestaña de Log in de SAP para los ERP's 
        subprocess.Popen(path)
        hwnd = 0
        start_time = time.time()
        while not hwnd:
            hwnd = win32gui.FindWindow(None, 'SAP Logon 770')
            if time.time() - start_time > 30:
                return  # Si se supera el tiempo máximo de espera, se sale de la función
            time.sleep(0.5) 

        # ----Detecta la ventana de SAP
        SapGuiAuto = win32com.client.GetObject('SAPGUI')
        if not type(SapGuiAuto) == win32.CDispatch:
            return

        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == win32.CDispatch:
            SapGuiAuto = None
            return

    if n_windows == 1:

        # ----Detecta la ventana de SAP
        SapGuiAuto = win32com.client.GetObject('SAPGUI')
        if not type(SapGuiAuto) == win32.CDispatch:
            return

        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == win32.CDispatch:
            SapGuiAuto = None
            return

        # Obtiene la cantidad de conexiones activas
        num_connections = len(application.Children)
        print(num_connections)

        # Imprime la cantidad de conexiones activas
        print("Cantidad de conexiones activas: ", num_connections)

        # Imprime los nombres de los sistemas conectados
        for i in range(num_connections):
            print("Sistema conectado ", i+1, ": ", application.Children(i).Id)
            print("Sistema conectado ", i+1, ": ", application.Children(i).ContainerType)
            print("Sistema conectado ", i+1, ": ", application.Children(i).TypeAsNumber)
            

# ======================================== CONFIGURACION PARA LA EJECUCION DEL PROMARAMA ========================================
# ----Da los parametros iniciales de la ejecucion de la libreria para ejecutar la pantalla emergente y en su defecto el resto del programa
if __name__ == '__main__':
    
    root = tk.Tk()
    
    # ----La "r" es para que el path de la imagen no tome como caracteres especiales los slash "\" sino como texto
    #root.iconbitmap(r"C:\Program Files (x86)\Nokia\Reporte YRA2\Reporte_YRA2\nokia.ico")
    # ----Si se quiere ejecutar en el computador
    root.iconbitmap(r"C:\\Users\\migumart\\OneDrive - Nokia\Archivos personales\\Automatizacion Python\\Reporte YRA2 (P20)\\nokia.ico")
    
    input_form = InputForm(root)
    
    root.mainloop()
# ===============================================================================================================================

====INICIALIZACION DE -REPORTE YRA2-

====INICIALIZACION DE LA VENTANA EMERGENTE DE -DESCARGA YRA2-

----Se presionó el boton SUBMIT ____ Procede a inicar -open_sap.saplogin(variante,username, password)-

====FINALIZACION DE LA VENTANA EMERGENTE DE -DESCARGA YRA2-

Cantidad SAP LOGON: 1
3
Cantidad de conexiones activas:  3
Sistema conectado  1 :  /app/con[0]
Sistema conectado  1 :  True
Sistema conectado  1 :  11
Sistema conectado  2 :  /app/con[1]
Sistema conectado  2 :  True
Sistema conectado  2 :  11
Sistema conectado  3 :  /app/con[2]
Sistema conectado  3 :  True
Sistema conectado  3 :  11
